In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.impute import KNNImputer
import os
from itertools import combinations
import matplotlib.pyplot as plt
from scipy import interp
import statsmodels.api as sm

ImportError: cannot import name 'KNNImputer' from 'sklearn.impute' (C:\Users\Camil\anaconda3\lib\site-packages\sklearn\impute.py)

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold

def CV_train_stratified_lr(X_train,y_train, weights, l2_coef, interactions):
    """Function to train and assess through 5 folds cross validation a logistic regression model with different parameters.
    X_train : Matrix of features
    y_train : Vector of labels
    weights : Vector of weights used for the weighted loss function
    l2_coef : Vector of coefficient for ridge regularisation
    interactions : Add polynomial features to take into account interaction between variables in X_train.
    The metric used for assessment is F1 score.
    
    returns a list of F1 scores for each models, which are the average of F1 scores on each cross validation fold.
    """
    #if interactions != int and interactions != None:
        #print("interactions must be of type int or None")
    
    if type(interactions) == int:
        
        poly = PolynomialFeatures(interactions, interaction_only=True)
        X_train = poly.fit_transform(X_train)
    else:
        pass
    cv_res_list = []
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for i in l2_coef:
        for j in weights:
                
                
            results = []
            for train_ix, test_ix in kfold.split(X_train, y_train):
                train_X, test_X = X_train[train_ix], X_train[test_ix]
                train_y, test_y = y_train[train_ix], y_train[test_ix]
                clf = LogisticRegression(class_weight= j , penalty = "l2", C = i , random_state=Random_seed).fit(train_X, train_y.ravel())
                res = f1_score(test_y,clf.predict(test_X))
                results.append(res)
            mean = sum(results)/len(results)    
            cv_res_list.append([i,j,mean])
    return(cv_res_list)

In [ ]:
def CV_train_stratified_xgb(X_train,y_train, lr, n_estimator,reg_lambda ,max_depth):
    """Function to train and assess through 5 folds cross validation a gradient boosting classification model with different parameters.
    The metric used for assessment is F1 score.
    X_train : Matrix of features
    y_train : Vector of labels
    lr : Vector of learning rates
    n_estimators : Vector of amount of weak models
    reg_lambda : Vector of values for L2 regularisation
    max_depth : Maximum depth of weak learners 
    l2_coef : vector of coefficient for ridge regularisation
    interactions : add polynomial features to take into account interaction between variables in X_train. 
    
    returns a list of F1 scores for each models, which are the average of F1 scores on each cross validation fold.
    """
    cv_res_list = []
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    pos_weight = (len(y_train) - np.sum(y_train))/np.sum(y_train)
    #pos_weight = 20
    for i in lr:
        for j in n_estimator:
            for k in reg_lambda:
                for l in max_depth:
                    
                    results = []
                    for train_ix, test_ix in kfold.split(X_train, y_train):
                        train_X, test_X = X_train[train_ix], X_train[test_ix]
                        train_y, test_y = y_train[train_ix], y_train[test_ix]
                        clf = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, n_estimators = j, max_depth = l, learning_rate = i, scale_pos_weight = pos_weight, reg_lambda = k)
                        clf.fit(train_X, train_y.ravel())
                        res = f1_score(test_y,clf.predict(test_X))
                        results.append(res)
                    mean = sum(results)/len(results)    
                    cv_res_list.append([i,j,k,l,mean])
    return(cv_res_list)

In [ ]:
def CV_train_stratified_randomforest(X_train,y_train, n_estimator,min_samples_leaf,max_features ,max_depth, criterion = "gini"):
    """Function to train and assess through 5 folds cross validation a random forest classification model with different parameters.
    The metric used for assessment is F1 score.
    X_train : Matrix of features
    y_train : Vector of labels
    n_estimators : Vector of amount of trees
    min_samples_leaf : Vector the minimum number of samples required to be at a leaf node
    max_features : Vector of the number of features to consider when looking for the best split
    max_depth : Maximum depth of trees 
    l2_coef : vector of coefficient for ridge regularisation
    interactions : add polynomial features to take into account interaction between variables in X_train. 
    
    returns a list of F1 scores for each models, which are the average of F1 scores on each cross validation fold.
    """
    cv_res_list = []
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    pos_weight = (len(y_train) - np.sum(y_train))/np.sum(y_train)
    #pos_weight = 20
    for i in n_estimator:
        for j in min_samples_leaf:
            for k in max_features:
                for l in max_depth:
                    
                    results = []
                    for train_ix, test_ix in kfold.split(X_train, y_train):
                        train_X, test_X = X_train[train_ix], X_train[test_ix]
                        train_y, test_y = y_train[train_ix], y_train[test_ix]
                        clf = RandomForestClassifier(n_estimators = i, min_sample_leaf = j, max_features = k, max_depth = l)
                        clf.fit(train_X, train_y.ravel())
                        res = f1_score(test_y,clf.predict(test_X))
                        results.append(res)
                    mean = sum(results)/len(results)    
                    cv_res_list.append([i,j,k,l,mean])
    return(cv_res_list)

In [ ]:
def AUC_and_Perf(target, pred_probs, label,title, save):
    """Function to return the ROC curve, AUC and different relevant performance metrics for a model.
    
    target : vector of true labels
    pred_probs : Vector of model outputs \in [0,1]
    label : ROC curve label on plot
    title : Plot title
    save : To save the plot
  
    
    returns a dataframe with for each threshold : Sensitivity, Specificity, PPV, NPV, Accuracy, F1 score, Diagnostic odd ratio (DOR) and DOR confidence intervals.
    """
    
    
    fpr, tpr, thresholds = roc_curve(target, pred_probs)
    roc_auc = auc(fpr, tpr)
    # plot the roc curve for the model
    pyplot.plot([0,1], [0,1], linestyle='--', label='Random classifier')
    pyplot.plot(fpr, tpr, marker='.', label= label + str(" (AUC = %0.2f)") % roc_auc)
    # axis labels
    pyplot.xlabel('1 - Specificity')
    pyplot.ylabel('Sensitivity')
    pyplot.legend()
    # show the plot
    pyplot.title(title)
    if save == True:
        plt.savefig('ROC_curve' +" " + str(title) + '.png')
    pyplot.show()
    F1s = [] 
    ppvs = []
    npvs = []
    DORs = []
    ci_up_DORs =[]
    ci_low_DORs = []
    accuracies = []
    for i in thresholds:
        preds = (pred_probs>i)*1
        #print(preds)
        #print(target)
        print()
        tp = np.sum(((preds==1) & (target==1))*1)
        fp = np.sum(((preds==1) & (target==0))*1)
        tn = np.sum(((preds==0) & (target==0))*1)
        fn = np.sum(((preds==0) & (target ==1))*1)
        ppv = tp/(tp + fp)
        npv = tn/(tn + fn)
        accuracy = (tp + tn)/(tp + tn + fp + fn)
        F1 = tp/( tp + 0.5 * (fp + fn))
        DOR = (tp * tn)/(fp * fn)
        lnDOR = np.log(DOR)
        SE_lnDOR = np.sqrt(1/tp + 1/fn + 1/fp + 1/tn)
        ci_up_DOR = np.exp(lnDOR + 1.96 * SE_lnDOR)
        ci_low_DOR = np.exp(lnDOR - 1.96 * SE_lnDOR)
        ci_up_DORs.append(ci_up_DOR)
        ci_low_DORs.append(ci_low_DOR)
        DORs.append(DOR)
        F1s.append(F1)
        ppvs.append(ppv)
        npvs.append(npv)
        accuracies.append(accuracy)
    perf_data = pd.DataFrame(list(zip(thresholds, tpr, 1-fpr,ppvs,npvs,accuracies,F1s,DORs,ci_up_DORs,ci_low_DORs)),
                   columns =['Threshold', 'Sensitivity',"Specificity","PPV","NPV", "Accuracy", "F1 score","DOR","CI_up","CI_low"])
    return perf_data

In [ ]:
def best_threshold(data, min_se = None, min_sp = None):
      """Function to return the best threshold with a minimum sensitivity or specificity.
    
    data : a dataframe with "Sensitivity" and "Specificity" column names.
    min_se : minimum wanted sensitivity
    min_sp : minimum wanted specificity
  
    
    returns a threshold value that maximises sensitivity and specificity with the wanted conditions
    """
    if min_se == None:
         res = data.loc[data[['Sensitivity','Specificity']].sum(1).idxmax()]
    else:
        data = data.loc[(data.Sensitivity>=  min_se) & (data.Specificity>= min_sp)]
        res = data.loc[data[['Sensitivity','Specificity']].sum(1).idxmax()]
    return res

In [ ]:
def combined_roc_AUC_CI(pred_probs1,pred_probs2, targets):
    """Function to return the ROC curve, AUC and different relevant performance metrics for the rule based model combining 2 positions.
    Also computes ROC curve of AUC by bootstrap.
    targets : vector of true labels
    pred_probs1 : Vector of position1 model outputs \in [0,1]
    pred_probs2 : Vector of position2 model outputs \in [0,1]
    
  
    
    returns a dataframe with for each threshold : ROC curves, AUC and boostrapped confidence intervals.
    """
    
    final_df = pd.DataFrame()
    dframe = pd.concat([pred_probs1,pred_probs2,targets], axis = 1)
    dframe = dframe.dropna(axis = "rows")
    posA = dframe.columns[0]
    posB=dframe.columns[1]
    #targets = dframe.iloc[:,[2]]
    n_bootstraps = 10000
    rng_seed = 42  # control reproducibility
    bootstrapped_scores = []
    rng = np.random.RandomState(rng_seed)
    for i in range(n_bootstraps):
        sensitivities = []
        specificities = []
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(dframe), len(dframe))
        df = dframe.iloc[indices,:]
        for t in np.linspace(0,1,101):
            targets = df.iloc[:,2]
            preds1 = (df.iloc[:,0]>t)*1
            preds2 = (df.iloc[:,1]>t)*1
            preds = preds1 + preds2
            preds = np.where(preds==2,1,preds)
            tp = np.sum(((preds==1) & (targets==1))*1)
            fp = np.sum(((preds==1) & (targets==0))*1)
            tn = np.sum(((preds==0) & (targets==0))*1)
            fn = np.sum(((preds==0) & (targets ==1))*1)
            sensitivity = tp/(tp + fn)
            specificity = tn/(tn + fp)
            sensitivities.append(sensitivity)
            specificities.append(specificity)
        tpr = np.array(sensitivities) 
        fpr = 1 - np.array(specificities)
        score = metrics.auc(fpr, tpr)
        bootstrapped_scores.append(score)       
    return bootstrapped_scores

In [ ]:
def combined_perf_thresholds(pred_probs, target):
    """Function to return the best combination of 2 positions for the rule based approach, for each combination, returns the
    threshold associated with the best performance in sensitivity and specificity.
    pred_probs : dataframe of probabilities
    target : Vector true labels
    returns a dataframe with for combination of position, the best performance and associated threshold.
    """
    
    best_threshold_df1 = pd.DataFrame()
    best_threshold_df2 = pd.DataFrame()
    final_df = pd.DataFrame()
    comb = combinations(pred_probs.columns,2)
    for i in comb:
        df = pred_probs.loc[:,i]
        df = pd.concat([df,target], axis = 1)
        df = df.dropna(axis = "rows")
        posA = df.columns[0]
        posB=df.columns[1]
        targets = df.iloc[:,2]
        thresh = []
        sensitivities = []
        specificities = []
        ppvs = []
        npvs = []
        accuracies = []
        for t in np.linspace(0,1,101):
            preds1 = (df.iloc[:,0]>t)*1
            preds2 = (df.iloc[:,1]>t)*1
            preds = preds1 + preds2
            preds = np.where(preds==2,1,preds)
            tp = np.sum(((preds==1) & (targets==1))*1)
            fp = np.sum(((preds==1) & (targets==0))*1)
            tn = np.sum(((preds==0) & (targets==0))*1)
            fn = np.sum(((preds==0) & (targets ==1))*1)
            sensitivity = tp/(tp + fn)
            specificity = tn/(tn + fp)
            ppv = tp/(tp + fp)
            npv = tn/(tn + fn)
            accuracy = (tp + tn)/(tp + tn + fp + fn)
            thresh.append(t)
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            ppvs.append(ppv)
            npvs.append(npv)
            accuracies.append(accuracy)
        perf_data = pd.DataFrame(list(zip(thresh, sensitivities, specificities,ppvs,npvs,accuracies)),
            columns =[str(posA) + " or " + str(posB) +' Threshold' +", " + "N = "+str(len(targets)),str(posA) + " or " + str(posB) +' Sensitivity',str(posA) + " or " + str(posB) + " Specificity",str(posA) + " or " + str(posB) + " PPV", str(posA) + " or " + str(posB) + " NPV", str(posA) + " or " + str(posB) + " Accuracy" ])
        best_thresh1 = perf_data.loc[perf_data.iloc[:,[1,2]].sum(1).idxmax()].to_frame().transpose().reset_index(drop=True)
        #print(best_thresh1)
        try:
            data_thresh = perf_data[(perf_data.iloc[:,1]>=  0.8) & (perf_data.iloc[:,2]>= 0.7)]
            best_thresh2 = data_thresh.loc[data_thresh.iloc[:,[1,2]].sum(1).idxmax()].to_frame().transpose().reset_index(drop=True)
        except:
            try:
                data_thresh = perf_data[(perf_data.iloc[:,1]>=  0.8) & (perf_data.iloc[:,2]>= 0.6)]
                best_thresh2 = data_thresh.loc[data_thresh.iloc[:,[1,2]].sum(1).idxmax()].to_frame().transpose().reset_index(drop=True)
            except:
                try: 
                    data_thresh = perf_data[(perf_data.iloc[:,1]>=  0.8) & (perf_data.iloc[:,2]>= 0.5)]
                    best_thresh2 = data_thresh.loc[data_thresh.iloc[:,[1,2]].sum(1).idxmax()].to_frame().transpose().reset_index(drop=True)
                except:
                    try:
                        data_thresh = perf_data[(perf_data.iloc[:,1]>=  0.8) & (perf_data.iloc[:,2]>= 0.4)]
                        best_thresh2 = data_thresh.loc[data_thresh.iloc[:,[1,2]].sum(1).idxmax()].to_frame().transpose().reset_index(drop=True)
                    except:
                        pass
            
        
        best_threshold_df1 = pd.concat([best_threshold_df1,best_thresh1],axis = 1)
        print(best_threshold_df1)
        best_threshold_df2 = pd.concat([best_threshold_df2,best_thresh2],axis = 1)
        final_df = pd.concat([final_df,perf_data], axis = 1)
    #final_df = final_df.iloc[:,0:(int(final_df.shape[1]/2))]        
    return best_threshold_df1, best_threshold_df2, final_df